# Itinerary planning

Based on the results of the vacation search script, this conducts the final step of allowing users to select an eligible country, identify four cities within that country, and map an optimal itinerary for those four cities using the Google Directions API.

In [1]:
import pandas as pd
import numpy as np
import os
import gmaps
import requests
import sys
import string

from pprint import pprint

sys.path.insert(0,'../')
from config import g_key

gmaps.configure(g_key)

In [2]:
vacation_cities_df = pd.read_csv(os.path.join("..","vacation_search","WeatherPy_vacation.csv")).drop(["Unnamed: 0"], axis=1)
vacation_cities_df

,City,Country,Lat,Long,Max Temp,Pct Humidity,Pct Cloudiness,Wind Speed,Description,Hotel Name
0,Lufilufi,WS,-13.8745,-171.5986,78.58,76,47,19.82,light rain,Leuaina Beach Resort and Spa
1,Paris,US,33.6609,-95.5555,71.17,83,75,10.36,broken clouds,"Holiday Inn Express & Suites Paris, an IHG Hotel"
2,Vaini,TO,-21.2000,-175.2000,77.16,100,90,13.80,moderate rain,Keleti Beach Resort
3,Garowe,SO,8.4054,48.4845,69.53,83,8,12.77,clear sky,Curubo Hotel
4,Kapaa,US,22.0752,-159.3190,78.19,85,75,1.01,light rain,Sheraton Kauai Coconut Beach Resort
...,...,...,...,...,...,...,...,...,...,...
192,Harper,LR,4.3750,-7.7169,79.56,83,54,8.70,broken clouds,Screensaver
193,Road Town,VG,18.4167,-64.6167,78.89,65,75,12.66,broken clouds,"ZINGARA 76ft Catamaran, full crew-all included"
194,Santa Maria,CV,16.6000,-22.9000,74.95,74,63,15.17,broken clouds,Porta do Vento
195,Chimoio,MZ,-19.1164,33.4833,68.92,99,100,3.27,moderate rain,Predio Livraria Ahmed


### Load all cities into google maps to display range of options

In [3]:
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Today's Weather</dt><dd>{Max Temp} and {Description}</dd>
<dt>Hotel Name</dt><dd>{Hotel Name}
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in vacation_cities_df.iterrows()]

coordinates_df = vacation_cities_df[["Lat","Long"]]
coordinates_df

fig = gmaps.figure(center=(0.0, 0.0), 
                   zoom_level=1.5,
                   layout = {'width': '800px',
                             'height': '500px'})

markers = gmaps.marker_layer(locations=coordinates_df, info_box_content=hotel_info)
fig.add_layer(markers)

## Call the figure
fig

Figure(layout=FigureLayout(height='500px', width='800px'))

### Generate user-friendly display of eligible countries for vacation

Merge data frame with existing lookups of ISO Code to full country name, and eliminate any country with less than 4 eligible cities.

In [4]:
country_code_map_df = pd.read_csv(os.path.join("..","weather_database","country_code_map.csv"))

country_counts_df = pd.DataFrame(data=vacation_cities_df.value_counts("Country"))

country_counts_df.columns = ["City Count"]

eligible_countries_df = pd.merge(left=country_counts_df,
                                 right=country_code_map_df,
                                 left_on = "Country",
                                 right_on = "Code",
                                 how="inner")

eligible_countries_df = eligible_countries_df[["Code", "Name", "City Count"]].loc[(eligible_countries_df["City Count"] >= 4)].set_index("Code")

print("Eligible Countries for vacations are as follows:")
eligible_countries_df

Eligible Countries for vacations are as follows:


,Name,City Count
Code,,
ID,Indonesia,19
AU,Australia,17
BR,Brazil,14
US,United States,11
PE,Peru,9
ZA,South Africa,8
NZ,New Zealand,7
IN,India,6
MX,Mexico,6


### Choose country for vacation based on list above.

User must input ISO code from above list in order to proceed.

In [7]:
# Allow user to choose country.
chosen_country_code = input("Please choose a country by entering the CODE value from above list of eligible countries: ").upper()

while len(eligible_countries_df.loc[eligible_countries_df.index == chosen_country_code]) == 0:
    chosen_country_code = input("Invalid selection.  Please try again: ").upper()

chosen_country_name = eligible_countries_df.loc[eligible_countries_df.index == chosen_country_code]["Name"].get(key=0)
chosen_country_count = eligible_countries_df.loc[eligible_countries_df.index == chosen_country_code]["City Count"].get(key=0)

print(f"You have chosen {chosen_country_name}, which has {chosen_country_count} eligible cities: ")

chosen_country_cities_df = vacation_cities_df.loc[vacation_cities_df["Country"] == chosen_country_code]
chosen_country_cities_df.reset_index(drop=True, inplace=True)
chosen_country_cities_df

Please choose a country by entering the CODE value from above list of eligible countries:  AU


You have chosen Australia, which has 17 eligible cities: 


,City,Country,Lat,Long,Max Temp,Pct Humidity,Pct Cloudiness,Wind Speed,Description,Hotel Name
0,Hobart,AU,-42.8794,147.3294,70.27,80,20,1.01,few clouds,Mantra on Collins Hobart
1,New Norfolk,AU,-42.7826,147.0587,78.75,72,0,2.73,clear sky,The Shingles Riverside Cottages
2,Carnarvon,AU,-24.8667,113.6333,73.47,83,40,16.11,scattered clouds,Hospitality Carnarvon
3,Moranbah,AU,-22.0016,148.0466,77.16,69,0,12.57,clear sky,Direct Hotels - Monterey Moranbah
4,Flinders,AU,-34.5833,150.8552,73.20,79,43,5.99,scattered clouds,Shellharbour Resort & Conference Centre
5,Port Lincoln,AU,-34.7333,135.8667,78.13,49,97,10.78,overcast clouds,Port Lincoln Hotel
6,Yulara,AU,-25.2406,130.9889,73.24,53,16,6.91,few clouds,Desert Gardens Hotel - Ayers Rock Resort
7,Portland,AU,-38.3333,141.6000,65.50,70,100,4.68,overcast clouds,Portland Holiday Village
8,Moree,AU,-29.4667,149.8500,73.42,64,58,9.22,broken clouds,Molika Springs Motel
9,Geraldton,AU,-28.7667,114.6000,66.61,88,75,3.44,broken clouds,Broadwater Mariner Resort


### City decision engine

User must now choose four cities, using the index (ID) column, to spend on their vacation.  These will then be loaded into dataframes for mapping into the Google API.

In [9]:
chosen_city_ids = list()

city_id_1 = int(input("Using the number value on the far left, please choose a starting/ending city for your trip: "))

while len(chosen_country_cities_df.loc[chosen_country_cities_df.index == city_id_1]) == 0:
    city_id_1 = int(input("Invalid selection.  Please try again: "))
    
chosen_city_ids.append(city_id_1)
    
vacation_start = chosen_country_cities_df.iloc[[city_id_1]]
vacation_end = chosen_country_cities_df.iloc[[city_id_1]]
city = vacation_start["City"].iloc[0]

print(f"You have chosen {city}.")

vacation_start

for i in range(3):
    city_id = int(input(f"Please identify city #{i+2} on your itinerary: "))
    
    while ((len(chosen_country_cities_df.loc[chosen_country_cities_df.index == city_id]) == 0) or (city_id in chosen_city_ids)):
        if (city_id in chosen_city_ids):
            temp_df = chosen_country_cities_df.iloc[city_id]
            city_name = temp_df["City"]
            city_id = int(input(f"You have already chosen {city_name}.  Please try again: "))
        else:
            city_id = int(input("Invalid selection.  Please try again: "))

    chosen_city_ids.append(city_id)    
   
    temp_df = chosen_country_cities_df.iloc[city_id]
    city_name = temp_df["City"]
    print(f"You have chosen {city_name}.")
        
    # This is ugly and ought to be refactored.
    if (i + 2 == 2):
        vacation_stop1 = chosen_country_cities_df.iloc[[city_id]]
    elif (i + 2 == 3):
        vacation_stop2 = chosen_country_cities_df.iloc[[city_id]]    
    elif (i + 2 == 4):
        vacation_stop3 = chosen_country_cities_df.iloc[[city_id]]    

Using the number value on the far left, please choose a starting/ending city for your trip:  14


You have chosen Port Macquarie.


Please identify city #2 on your itinerary:  1


You have chosen New Norfolk.


Please identify city #3 on your itinerary:  13


You have chosen Emerald.


Please identify city #4 on your itinerary:  16


You have chosen Mareeba.


In [10]:
# Get the lat-long pairs
start = vacation_start[["Lat", "Long"]].to_numpy()
end = vacation_end[["Lat", "Long"]].to_numpy()
stop1 = vacation_stop1[["Lat", "Long"]].to_numpy()
stop2 = vacation_stop2[["Lat", "Long"]].to_numpy()
stop3 = vacation_stop3[["Lat", "Long"]].to_numpy()

### Map itinerary.

Pass all four cities into the Google Directions API and generate an itinerary, allowing the API to optimize the route between cities.

In [11]:
fig = gmaps.figure()
chosen_trip = gmaps.directions_layer(
         start=(start[0][0],start[0][1]),
         end=(end[0][0],end[0][1]),
         waypoints=[(stop1[0][0],stop1[0][1]),
                    (stop2[0][0],stop2[0][1]),
                    (stop3[0][0],stop3[0][1])],
        optimize_waypoints=True
)
fig.add_layer(chosen_trip)
fig

Figure(layout=FigureLayout(height='420px'))

### Display list of cities in itinerary and map their locations

In [12]:
itinerary_df = pd.concat(objs=[vacation_start, vacation_stop1, vacation_stop2, vacation_stop3],
                         ignore_index=True)
itinerary_df

,City,Country,Lat,Long,Max Temp,Pct Humidity,Pct Cloudiness,Wind Speed,Description,Hotel Name
0,Port Macquarie,AU,-31.4333,152.9167,70.90,73,92,6.96,overcast clouds,Rydges Port Macquarie
1,New Norfolk,AU,-42.7826,147.0587,78.75,72,0,2.73,clear sky,The Shingles Riverside Cottages
2,Emerald,AU,-23.5333,148.1667,78.35,73,0,12.66,clear sky,The Irish Village
3,Mareeba,AU,-17.0000,145.4333,78.44,94,13,2.42,few clouds,Trinity Plains Tourist Park


In [13]:
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Today's Weather</dt><dd>{Max Temp} and {Description}</dd>
<dt>Hotel Name</dt><dd>{Hotel Name}
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

coordinates_df = itinerary_df[["Lat","Long"]]
coordinates_df

fig = gmaps.figure(layout = {'width': '800px',
                             'height': '500px'})

markers = gmaps.marker_layer(locations=coordinates_df, info_box_content=hotel_info)
fig.add_layer(markers)

## Call the figure
fig

Figure(layout=FigureLayout(height='500px', width='800px'))